# OpenAI API Scoring Test
This notebook runs the `OpenAIZeroShotScorer` with sample context and candidate POIs to verify the JSON response from the OpenAI Chat Completions API.

## 1. Prerequisites
- Store your `OPENAI_API_KEY` in the project root `.env` file.
- Execute the cells in order to load environment variables, assemble sample data, and call the API.

In [1]:
# 1) Load environment variables from .env
from pathlib import Path
import os

env_path = Path.cwd() / '.env'
print(env_path)
if not env_path.exists():
    raise FileNotFoundError('Missing .env file with OPENAI_API_KEY')

for line in env_path.read_text(encoding='utf-8').splitlines():
    line = line.strip()
    if not line or line.startswith('#') or '=' not in line:
        continue
    key, value = line.split('=', 1)
    os.environ[key.strip()] = value.strip()

print('Environment variables loaded successfully.')

c:\Users\7info\Desktop\연구\Mobile-Recommendation-on-device\.env
Environment variables loaded successfully.


In [1]:
# 2) Import project modules and build sample data
import sys
from datetime import datetime
from pathlib import Path

project_root = Path.cwd()
src_path = project_root / 'src'
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

from glp_demo.openai_scorer import OpenAIZeroShotScorer
from glp_demo.demo_data import create_sample_events, create_sample_pois
from glp_demo.pkg import PersonalKnowledgeGraph
from glp_demo.process_mining import build_routine_model
from glp_demo.app_signals import infer_intent_signals

events = create_sample_events()
pois = create_sample_pois()
routine = build_routine_model(events)
pkg = PersonalKnowledgeGraph()
pkg.ingest_events(events)
pkg.link_routine(routine)

reference_time = datetime(2025, 10, 3, 18, 30)
intents = infer_intent_signals(events, reference_time)

zero_shot_pois = [poi for poi in pois if pkg.place_visit_counts.get(poi.poi_id, 0) == 0]
poi_features = []
for poi in zero_shot_pois:
    poi_features.append({
        'poi_id': poi.poi_id,
        'name': poi.name,
        'category': poi.category,
        'reasoning_tokens': {
            'routine': 0.2,
            'personal': 0.0,
            'intent': 0.7,
            'distance': 0.8,
        },
    })

context = {
    'time_slot': 'evening',
    'intent_scores': {intent.intent: intent.score for intent in intents[:5]},
    'reference_time': reference_time.isoformat(),
    'user_location': {'latitude': 36.0545, 'longitude': 129.3720},
}

print('Sample data prepared.')
print(f'Selected POIs: {len(poi_features)}')
for feature in poi_features:
    print(f"- {feature['name']} ({feature['category']})")

Sample data prepared.
Selected POIs: 18
- Premium Fitness Club (gym)
- Sunrise Yoga Studio (pilates)
- Ocean View Cafe (coffee_shop)
- Bibimbap House (korean_restaurant)
- Pasta Paradise (italian_restaurant)
- Cheonglim Pizza (italian_restaurant)
- Artisan Bakery & Cafe (bakery)
- 24/7 Convenience Store (convenience_store)
- Healthy Smoothie Bar (cafe)
- Zen Pilates Studio (pilates)
- Gangnam Fitness Center (gym)
- Hongdae Artisan Coffee (coffee_shop)
- Itaewon Italian Kitchen (italian_restaurant)
- Myeongdong Traditional House (korean_restaurant)
- Gangnam Luxury Cafe (coffee_shop)
- Seoul Station Convenience (convenience_store)
- Gangnam French Bakery (bakery)
- Gangnam Premium Pilates (pilates)


In [6]:
# 3) Call the OpenAI API and display the response
scorer = OpenAIZeroShotScorer(
    model='o3',
    temperature=0.1,
    max_tokens=500,
)

prompt = scorer._build_prompt(context, poi_features)
print('=' * 60)
print('PROMPT SENT TO OPENAI')
print('=' * 60)
print(prompt)
print('=' * 60)

scores = scorer.score(context, poi_features)

print('API RESPONSE (parsed scores)')
print('=' * 60)
for feature in poi_features:
    poi_id = feature['poi_id']
    score = scores.get(poi_id, 0.0)
    print(f"{feature['name']} ({feature['category']})")
    print(f"  POI ID: {poi_id}")
    print(f"  LLM Score: {score:.3f}")
    base = feature['reasoning_tokens']
    print(
        f"  Base signals: routine={base['routine']:.2f}, intent={base['intent']:.2f}, distance={base['distance']:.2f}"
    )
    print('-' * 40)

PROMPT SENT TO OPENAI
You are an on-device recommendation model assistant.
Given the user context and POI candidates, return a JSON object that maps each poi_id to a relevance score between 0 and 1.

Context:
- time_slot: evening
- reference_time: 2025-10-03T18:30:00
- user_location: {'latitude': 36.0545, 'longitude': 129.372}
- intents: meal:277.000, planning:133.500, social:101.500, exercise:76.000, commute:59.500

Candidates:
- new_premium_gym | name=Premium Fitness Club | category=gym | signals=routine=0.200, personal=0.000, intent=0.700, distance=0.800
- sunrise_yoga_studio | name=Sunrise Yoga Studio | category=pilates | signals=routine=0.200, personal=0.000, intent=0.700, distance=0.800
- ocean_view_cafe | name=Ocean View Cafe | category=coffee_shop | signals=routine=0.200, personal=0.000, intent=0.700, distance=0.800
- bibimbap_house | name=Bibimbap House | category=korean_restaurant | signals=routine=0.200, personal=0.000, intent=0.700, distance=0.800
- pasta_paradise | name=Pa

OpenAI API call failed: Error code: 403 - {'error': {'message': 'Project `proj_UktKInUHNllkg13RvQGzVGU4` does not have access to model `o3`', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_found'}}


API RESPONSE (parsed scores)
Premium Fitness Club (gym)
  POI ID: new_premium_gym
  LLM Score: 0.500
  Base signals: routine=0.20, intent=0.70, distance=0.80
----------------------------------------
Sunrise Yoga Studio (pilates)
  POI ID: sunrise_yoga_studio
  LLM Score: 0.500
  Base signals: routine=0.20, intent=0.70, distance=0.80
----------------------------------------
Ocean View Cafe (coffee_shop)
  POI ID: ocean_view_cafe
  LLM Score: 0.500
  Base signals: routine=0.20, intent=0.70, distance=0.80
----------------------------------------
Bibimbap House (korean_restaurant)
  POI ID: bibimbap_house
  LLM Score: 0.500
  Base signals: routine=0.20, intent=0.70, distance=0.80
----------------------------------------
Pasta Paradise (italian_restaurant)
  POI ID: pasta_paradise
  LLM Score: 0.500
  Base signals: routine=0.20, intent=0.70, distance=0.80
----------------------------------------
Cheonglim Pizza (italian_restaurant)
  POI ID: cheonglim_pizza
  LLM Score: 0.500
  Base signals